In [ ]:
"""This library queries the YouTube API for the list of videos on any given channel and returns the video ids, details,
and stats as a pandas dataframe that can be exported to a .csv"""

In [1]:
#Import necessary libraries and load api keys
import requests
import json
import pandas as pd

key_file_loc = "C:/Users/CodeAlias/Desktop/tweepy_auth.txt"

with open(key_file_loc) as f: keys = dict(x.rstrip().split(None,1) for x in f)
    
key = keys['ytapi']

channel_id = ["UC3XTzVzaHQEd30rQbuvCtTQ"]

In [3]:
for name in channel_id:
    url = "https://www.googleapis.com/youtube/v3/channels?part=contentDetails&id={}&key={}".format(name,key)
    response = requests.get(url)
    file_ = json.loads(response.text)
    playlist = file_['items'][0]['contentDetails']['relatedPlaylists']['uploads']

    #get list of all video ids from the list of uploads from a given channel
    if playlist:
        next_page_token = None
        ids = []
        run = True
        while run:
            query = "https://www.googleapis.com/youtube/v3/playlistItems?part=contentDetails&maxResults=50&playlistId={}&key={}".format(playlist, key)

            if next_page_token:
                query += "&pageToken={}".format(next_page_token)

            response = json.loads((requests.get(query)).text)

            for item in response['items']:
                ids.append(item['contentDetails']['videoId'])

            try: 
                next_page_token = response['nextPageToken']
            except:
                run = False

                # for all ids in the list, get stats and details

    df = pd.DataFrame()

    for id_ in ids:

        query = "https://www.googleapis.com/youtube/v3/videos?part=statistics,snippet&id={}&key={}".format(id_,key)
        response = json.loads((requests.get(query).text))

        dict_ = {
            'videoId' : id_,
            'channelId' : response['items'][0]['snippet']['channelId'],
            'channelTitle' : response['items'][0]['snippet']['channelTitle'],
            'description' : response['items'][0]['snippet']['description'],
            'title' : response['items'][0]['snippet']['title'],
            'publishedAt' : response['items'][0]['snippet']['publishedAt'],
            'commentCount' : response['items'][0]['statistics']['commentCount'],
            'dislikeCount' : response['items'][0]['statistics']['dislikeCount'],
            'likeCount' : response['items'][0]['statistics']['likeCount'],
            'viewCount' : response['items'][0]['statistics']['viewCount']       

        }

        df = df.append(dict_, ignore_index=True)

    df.to_csv("{}_video_stats.csv".format(name))


In [7]:
#get all comments from any video
#this currently only grabs what appears in the top comments and the replies to those comments

"""

IMPORTANT NOTE: THE API IS NOT RELIABLE FOR GRABBING COMMENTS

"""

comment_frame = pd.DataFrame()
ids = [ids[0]]
for id_ in ids:
    
    next_page_token = None
    run = True
    comments = []
    
    while run:
        query = "https://www.googleapis.com/youtube/v3/commentThreads?part=snippet&maxResults=100&videoId={}&key={}".format(id_,key)
        if next_page_token:
            query += "&pageToken={}".format(next_page_token)
        
        response = json.loads((requests.get(query)).text)
        
        for item in response['items']:
            comments.append(item)
            
        try: 
            next_page_token = response['next_page_token']
        except:
            run = False
    
    for item in comments:
        dict_ = {
            'id' : item['id'],
            'authorChannelUrl' : item['snippet']['topLevelComment']['snippet']['authorChannelUrl'],
            'authorDisplayName' : item['snippet']['topLevelComment']['snippet']['authorDisplayName'],
            'likeCount' : item['snippet']['topLevelComment']['snippet']['likeCount'],
            'publishedAt' : item['snippet']['topLevelComment']['snippet']['publishedAt'],
            'textDisplay' : item['snippet']['topLevelComment']['snippet']['textDisplay'],
            'videoId' : item['snippet']['topLevelComment']['snippet']['videoId'],
            'viewerRating' : item['snippet']['topLevelComment']['snippet']['viewerRating'],
            'totalReplyCount' : item['snippet']['totalReplyCount'],
            'isReply' : 0,
            'isReplyTo' : ""
        }
        comment_frame = comment_frame.append(dict_, ignore_index=True)
        
        if dict_['totalReplyCount'] > 0:
            replies = []
            next_page_token = None
            run = True
        
            while run:
                query = "https://www.googleapis.com/youtube/v3/comments?part=snippet&textFormat=plainText&maxResults=100&parentId={}&key={}".format(dict_['id'],key)
                
                if next_page_token:
                    query += "&pageToken={}".format(next_page_token)
                
                response = json.loads((requests.get(query)).text)
                
                for item in response['items']:
                    replies.append(item)
                
                try: 
                    next_page_token = response['nextPageToken']
                except:
                    run = False
            
            for item in replies:
                
                rdict_ = {
                    'id' : item['id'],
                    'authorChannelUrl' : item['snippet']['authorChannelUrl'],
                    'authorDisplayName' : item['snippet']['authorDisplayName'],
                    'likeCount' : item['snippet']['likeCount'],
                    'publishedAt' : item['snippet']['publishedAt'],
                    'textDisplay' : item['snippet']['textDisplay'],
                    'videoId' : dict_['videoId'],
                    'viewerRating' : item['snippet']['viewerRating'],
                    'totalReplyCount' : 0,
                    'isReply' : 1,
                    'isReplyTo' : dict_['authorDisplayName']
                }
                
                comment_frame = comment_frame.append(rdict_, ignore_index = True)
                
            
    

    

    